#### Question 1

Using the dataset, create Lasso regression models to predict the ‘mpg’ for a car 
using 20% and 80% of the training data for the 2 models.

In [245]:

#We will split the data into two - 20% and 80%. 
#We shall use each of these to test and train each other models

import pandas as pd

cars = pd.read_csv("assignment4.csv")

#Getting the dimensions of the dataset
print(cars.shape)

print(cars.head())


(398, 9)
   Unnamed: 0   mpg  cylinders  displacement  horsepower  weight  \
0           0  18.0          8         307.0       130.0    3504   
1           1  15.0          8         350.0       165.0    3693   
2           2  18.0          8         318.0       150.0    3436   
3           3  16.0          8         304.0       150.0    3433   
4           4  17.0          8         302.0       140.0    3449   

   acceleration  model year  origin  
0          12.0          70       1  
1          11.5          70       1  
2          11.0          70       1  
3          12.0          70       1  
4          10.5          70       1  


In [262]:
#Dropping the first column as it is not necessary

try:
    cars = cars.drop(columns = ['Unnamed: 0'])
except:
    pass


from sklearn.model_selection import train_test_split

#Creating the training data

# Splitting the data into 80-20 chunks
split_size=0.8

X = cars.drop(columns = ['mpg']).copy()
y = cars['mpg']

#Splitting the data into two chunks
X_80, X_20, y_80, y_20 = train_test_split(X,y, train_size=split_size, random_state= 75)

#print(X_80.shape)
#print(X_20.shape)
#print(y_80.shape)
#print(y_20.shape)

from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score #For calculating R-squared
from sklearn.metrics import mean_squared_error #For calculating MSE
##################################################################

#Fitting the model with 80% train data

#Defining the Lasso model with lambda (alpha in the method) = 1

model_80train_20test = Lasso(alpha=1.0)

model_80train_20test.fit(X_80,y_80)

#Predicting the output with the Out of sample data
y_80_predicted = model_80train_20test.predict(X_80)

#Calculating the R-squared
r2_80train_20_test = r2_score(y_80, y_80_predicted)

#Calculating the MSE
mse_80train_20_test = mean_squared_error(y_80, y_80_predicted)


###################################################################

#Fitting the model with 20% train data
    
#Defining the Lasso model with lambda (alpha in the method) = 1

model_20train_80test = Lasso(alpha=1.0)

model_20train_80test.fit(X_20,y_20)

#Predicting the output with the Out of sample data
y_20_predicted = model_20train_80test.predict(X_20)

#Calculating the Out-of-sample R-squared
r2_20train_80_test = r2_score(y_20, y_20_predicted)

#Calculating the Out-of-sample MSE
mse_20train_80_test = mean_squared_error(y_20, y_20_predicted)


print("The R squared for 80% trained model is", round(r2_80train_20_test,4))
print("\nThe MSE for 80% trained model is", round(mse_80train_20_test,4))
print("\nThe coefficients for 80% model are", model_80train_20test.coef_)
print("\nThe OOS R squared for 80% trained model is", round(r2_score(y_20, model_80train_20test.predict(X_20)),4))

print("\n\nThe R squared for 20% trained model is", round(r2_20train_80_test,4))
print("\nThe MSE for 20% trained model is", round(mse_20train_80_test,4))
print("\nThe coefficients for 20% model are", model_20train_80test.coef_)
print("\nThe OOS R squared for 20% trained model is", round(r2_score(y_80, model_20train_80test.predict(X_80)),4))


The R squared for 80% trained model is 0.7989

The MSE for 80% trained model is 12.2428

The coefficients for 80% model are [ 0.          0.         -0.00775594 -0.00656387  0.          0.64923279
  0.        ]

The OOS R squared for 80% trained model is 0.8336


The R squared for 20% trained model is 0.8413

The MSE for 20% trained model is 9.7054

The coefficients for 20% model are [-0.          0.          0.00893342 -0.00677418  0.04029953  0.7460807
  0.        ]

The OOS R squared for 20% trained model is 0.7986


#### Question 2

For the above model, tune ‘alpha’ using the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC) to find the optimum value for it. Explain how that value for alpha was chosen. Build a model using that value of alpha.

As we see, the model with 80% training data has better OOS R squared as well as MSE. Hence we will choose this model.

In [247]:
from sklearn.preprocessing import StandardScaler #For standardizing
#I will not use in this example

from sklearn.linear_model import LassoLarsIC #For tuning alpha (lambda)
from sklearn.pipeline import make_pipeline 

import time #Optional

lasso_lars_ic = make_pipeline(
    LassoLarsIC(criterion="aic", normalize=False)
).fit(X_80, y_80)


#Calculating AIC for alphas
results = pd.DataFrame(
    {
        "alphas": lasso_lars_ic[-1].alphas_,
        "AIC criterion": lasso_lars_ic[-1].criterion_,
    }
).set_index("alphas")
alpha_aic = lasso_lars_ic[-1].alpha_

#Calculating BIC for alphas

lasso_lars_ic.set_params(lassolarsic__criterion="bic").fit(X_80, y_80) #Changing to bic
results["BIC criterion"] = lasso_lars_ic[-1].criterion_
alpha_bic = lasso_lars_ic[-1].alpha_

#Putting the results in a tabular form
def highlight_min(x):
    x_min = x.min()
    return ["font-weight: bold" if v == x_min else "" for v in x]


results.style.apply(highlight_min)

#reference: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html


,AIC criterion,BIC criterion
alphas,,
5396.386629286816,318.000000,318.000000
25.612232052031285,101.785670,105.547722
15.803221689276588,102.623031,110.147133
8.810170762726942,102.471467,113.757621
2.532008485512097,72.223262,83.509416
0.8389715747340596,69.826350,81.112504
0.4436366205191914,71.591201,86.639406
0.08902377684326214,70.688751,89.499008
0.04761791623217235,72.591364,95.163672


As we see, for an alpha of 0.8389, both the AIC and the BIC are the least. This indicates that the model has the least in-sample deviance for a hyperparameter value of 0.8389.

For each increasing value of alpha, the penalty function increases - indicating that lesser variables have a chance to get selected. So when we tune alpha using AIC or BIC, we select a value that gives us the minimum "deviance from truth".

In [248]:
#Building a model with the chosen value of alpha

#Defining the Lasso model with lambda (alpha in the method) = 0.8389

model_low_aic_bic = Lasso(alpha= 0.8389715747340596)

model_low_aic_bic.fit(X_80,y_80)
print("\nThe coefficients for the model with low AIC/BIC are", model_low_aic_bic.coef_)

#This is the model with the new hyperparameter alpha

predicted_y_low_aic_bic = model_low_aic_bic.predict(X_80)
r2_low_aic_bic = r2_score(y_80, predicted_y_low_aic_bic)

#Calculating the Out-of-sample MSE
mse_low_aic_bic = mean_squared_error(y_80, predicted_y_low_aic_bic)

#Calculating the SSE
sse_low_aic_bic = mse_low_aic_bic*len(y_80)

print("\nThe out of sample R-square for the model with lowest AIC/BIC is", round(r2_low_aic_bic,4))


The coefficients for the model with low AIC/BIC are [ 0.00000000e+00  8.42099254e-06 -7.44427408e-03 -6.55919900e-03
  0.00000000e+00  6.62518239e-01  0.00000000e+00]

The out of sample R-square for the model with lowest AIC/BIC is 0.7993


#### Question 3

Build a simple regression model using 10-fold cross validation for the same data. Write your observations about the R-squared values for the models and their predictions.

In [258]:
#For cross validation, we will use the entire data set instead of the 80-20 split

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np

#Defining the CV method to use. 10-fold
cv = KFold(n_splits=10, random_state=28, shuffle=True)

#Defining a linear regression model
model_cv = LinearRegression()

#use k-fold CV to evaluate model
cv_r_squares = cross_val_score(model_cv, X, y, scoring='r2',
                         cv=cv, n_jobs=-1)
print("The R-squared for the Cross Validation model is",round(np.mean(cv_r_squares),4))


The R-squared for the Cross Validation model is 0.8092


The linear regression with cross validation has an R-squared of 80.92%.

The Out of Sample R squared value for the 80% trained model Lasso (alpha = 1) is 83.36% and that of the 20% trained model is 79.89%.

This shows that in this case, Lasso model with alpha = 1 and 80% training data has the highest out of sample R-squared.

#### Question 4
Calculate AIC, AICc (i.e. Corrected AIC) and BIC for the models you built in question 1 and question 2. According to each of the measures, which is the better model? Is BIC always greater than AIC? Please explain. Compare the AICs with the corresponding corrected AICs.

In [255]:
#Following are the 3 models built in Questions 1 & 2

#model_80train_20test
#model_20train_80test
#model_low_aic_bic

def info_criteria(n, mse, num_params):
    aic = n * log(mse) + 2 * num_params
    aic_c = aic + (2*num_params*(num_params+1))/(n - num_params - 1)
    bic = n * log(mse) + log(n) * num_params
    return ({"AIC":aic}, {"BIC":bic}, {"AICc" : aic_c})


#Model with 80% train data
print(info_criteria(len(X_80), mse_80train_20_test, 3))

#Model with 20% train data
print(info_criteria(len(X_20), mse_20train_80_test, 4))

#Model selected after tuning alpha
print(info_criteria(len(X_80), mse_low_aic_bic, 4))


({'AIC': 802.5706613185764}, {'BIC': 813.8568154669169}, {'AICc': 802.6470944395955})
({'AIC': 189.81491600472398}, {'BIC': 199.34302254341952}, {'AICc': 190.34824933805731})
({'AIC': 803.950392058235}, {'BIC': 818.9985975893558}, {'AICc': 804.0781875853916})


Although the AIC, BIC and AICc for the second model (With 20% train data) are lesser, we cannot compare these with the values for the other two models as the data used for training the 80% models is different from the one used for training the 20% model.

Information Criteria cannot be compared between models built from different datasets.

In the two models using the 80% train data, the model with alpha = 1 gives a better BIC (of 813.85) compared to the model with alpha = 0.83 (BIC = 818.99). Hence, with this training set, it is better to choose a hyperparameter (alpha) = 1. The AIC and AICc values are lower as well.

The values of BIC are greater than AIC because BIC penalizes addition of new parameters more strongly compared to AIC. BIC is parsimony sensitive. AIC always has a chance of choosing too big a model, regardless of number of rows. BIC has very little chance of choosing too big a model if number of variables is sufficient, but it has a larger chance than AIC, for any given n, of choosing too small a model. Hence, the penalty for choosing a larger model for BIC is higher than that of AIC.

#### Question 5

What are Randomized Control Trials: A/B Tests? What is its significance? Elaborate. Explain methods used in analyzing the RCT data?


In Randomized Control Trials, two or more parts of the population are randomly picked and a treatment (which is to be tested) is applied to one of the parts. 

A/B tests are widely used in multiple domains, especially in marketing analytics. They are used by companies to test new features in their websites when they cannot afford to roll it out to the whole population. They are also used to test two different types of creatives in the advertising domain.

The most important aspect before performing an RCT is to ensure that the two populations are randomly selected as not randomizing will affect the results. The treatment effect is given by the difference in the response of interest between the population receiving the treatment versus the population not receiving the treatment.

This difference will help estimate the actual impact that the treatment had on the population when compared to the control.

Active Learning (AL) one of the methods that is used in analyzing RCT data. In AL, the outcome variable is constantly monitored for the RCT across all the treatments and the treatments giving the best results are given more weightage than the others. This enables quick selection of the best performing treatment. This is also known as Multi Armed bandits - and is an extremely popular technique used in marketing.

Block-Design Experiment is also a method of performing and analyzing RCT data. In this design, the population is divided into blocks based on external variables (https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-32833-1_344). Each of these block receives the treatment and the control at some point. The treatment effect is then calculated by summing the differences in the outcome for treated and untreated across all blocks and then taking the average of this. This treatment effect can be interpreted as a causal effect as by using the block-design, other variables have been controlled for.